# **API & WEBHOOK-BASED DATA RETRIEVAL**

### **1) REQUIREMENTS SETUP**

In [1]:
# !pip install -r requirements.txt

In [50]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import numpy as np

### **2) MODULES IMPORT**

In [3]:
from FRED_module import fetch_FRED
from EUROSTAT_module import fetch_EUROSTAT
from WB_module import fetch_WB 
from YFINANCE_module import fetch_YFINANCE
from DBNOMICS_module import fetch_DBNOMICS

### **3) DATA FETCHING**

#### **3.1) EUROSTAT-extracted indicators**

In [4]:
# Export to the US by EU member state (monthly, total, millions of EUR)
# EU27 (from 2020) trade by SITC product group (Trade value in million ECU/EURO, Standard International Trade Classification (SITC Rev. 4, 2006): Total - all products)
# https://ec.europa.eu/eurostat/databrowser/view/ds-059331/legacyMultiFreq/table?lang=en 

# data before 2025 before tariffs
# EURO_expus_m_raw = 

In [5]:
# Industrial Production by EU member state (monthly, total manufacturing, Index: 2021=100, non-seasonally adjusted, 1996-01, 2025-08)
# https://ec.europa.eu/eurostat/databrowser/view/sts_inpr_m/default/table?lang=en
EURO_indprod_m_raw = fetch_EUROSTAT(
    "sts_inpr_m",
    filters = {
    "geo": ["EU27_2020", "EU28", "EA20", "EA19",  
    "BE", "BG", "CZ", "DK", "DE", "EE", "IE", "EL", "ES", "FR",
    "HR", "IT", "CY", "LV", "LT", "LU", "HU", "MT", "NL", "AT",
    "PL", "PT", "RO", "SI", "SK", "FI", "SE"],
    "s_adj": "NSA",
    "unit": "I21",
    "nace_r2": "C"
    }
)

EURO_indprod_m_raw = EURO_indprod_m_raw[[ "geo", "time", "value" ]]
EURO_indprod_m_raw = EURO_indprod_m_raw.rename(columns={
    "geo": "Country",
    "time": "Time",
    "value": "Indprod Index (I21)"
})

EURO_indprod_m_raw.tail()

,Country,Time,Indprod Index (I21)
8971,SK,2025-04,102.5
8972,SK,2025-05,101.8
8973,SK,2025-06,100.2
8974,SK,2025-07,87.7
8975,SK,2025-08,86.4


In [6]:
# Unemployment rate by EU member state (monthly, Percentage of population in the labour force (age-class = total, sex-class=total), non-seasonally adjusted, 1994-01 (varying), 2025-08)
# https://ec.europa.eu/eurostat/databrowser/view/une_rt_m/default/table?lang=en
EURO_unem_m_raw = fetch_EUROSTAT(
    "une_rt_m",
    filters = {
    "geo": ["EU27_2020", "EU28", "EA20", "EA19",  
    "BE", "BG", "CZ", "DK", "DE", "EE", "IE", "EL", "ES", "FR",
    "HR", "IT", "CY", "LV", "LT", "LU", "HU", "MT", "NL", "AT",
    "PL", "PT", "RO", "SI", "SK", "FI", "SE"],
    "s_adj": "NSA",
    "unit": "PC_ACT"
    }
)

EURO_unem_m_raw = EURO_unem_m_raw[[ "geo", "time", "value" ]]
EURO_unem_m_raw = EURO_unem_m_raw.rename(columns={
    "geo": "Country",
    "time": "Time",
    "value": "Unemployment Rate (%pop in LF)"
})

EURO_unem_m_raw.tail()

,Country,Time,Unemployment Rate (%pop in LF)
98345,SK,2025-04,13.9
98346,SK,2025-05,14.3
98347,SK,2025-06,15.4
98348,SK,2025-07,15.9
98349,SK,2025-08,15.8


In [ ]:
# GDP

In [ ]:
# Inflation

#### **3.2) FRED-extracted indicators**

In [8]:
# US-EUR exchange rate, U.S. Dollars to One Euro (initially daily, converted to monthly, non-seasonally adjusted, 1999-01, 2025-10)
# https://fred.stlouisfed.org/series/DEXUSEU
EXUSEU_d_raw = fetch_FRED("DEXUSEU") 
EXUSEU_d_raw = EXUSEU_d_raw.rename(columns= 
        {"date": "Time", 
         "DEXUSEU": "USD-EUR Spot Exchange Rate"
})

# Aggregation (dimension from daily to monthly)
EXUSEU_d_raw = EXUSEU_d_raw.set_index("Time")
EXUSEU_m_raw = EXUSEU_d_raw.resample("M").mean().reset_index()
EXUSEU_m_raw["Time"] = EXUSEU_m_raw["Time"].dt.to_period("M").astype(str)

EXUSEU_m_raw.tail()

,Time,USD-EUR Spot Exchange Rate
317,2025-06,1.153365
318,2025-07,1.167082
319,2025-08,1.164748
320,2025-09,1.173871
321,2025-10,1.164894


In [36]:
# US-BGN exchange rate, U.S. Dollars to One Bulgarian Lev (monthly, non-seasonally adjusted, 1960-01, 2021-06)
# https://fred.stlouisfed.org/series/BGRCCUSMA02STM

EXUSBGN_m_raw = fetch_FRED("BGRCCUSMA02STM") 
EXUSBGN_m_raw = EXUSBGN_m_raw.rename(columns= 
        {"date": "Time", 
         "BGRCCUSMA02STM": "USD-BGN Spot Exchange Rate"
})

EXUSBGN_m_raw["Time"] = EXUSBGN_m_raw["Time"].dt.to_period("M").astype(str)

EXUSBGN_m_raw["Country"] = "BG"
EXUSBGN_m_raw = EXUSBGN_m_raw[["Country", "Time", "USD-BGN Spot Exchange Rate"]]

EXUSBGN_m_raw.tail()

,Country,Time,USD-BGN Spot Exchange Rate
769,BG,2021-02,1.6167
770,BG,2021-03,1.6447
771,BG,2021-04,1.6366
772,BG,2021-05,1.6084
773,BG,2021-06,1.6237


In [35]:
# US-SKK exchange rate, U.S. Dollar to One Swedish Kronor (initially SKK-US daily, converted to US-SKK monthly, non-seasonally adjusted, 1971-01, 2025-10)
# https://fred.stlouisfed.org/series/DEXSDUS
EXSKKUS_d_raw = fetch_FRED("DEXSDUS") 
EXSKKUS_d_raw = EXSKKUS_d_raw.rename(columns= 
        {"date": "Time", 
         "DEXSDUS": "SKK-USD Spot Exchange Rate"
})

# Aggregation (dimension from daily to monthly)
EXSKKUS_d_raw = EXSKKUS_d_raw.set_index("Time")
EXSKKUS_m_raw = EXSKKUS_d_raw.resample("M").mean().reset_index()
EXSKKUS_m_raw["Time"] = EXSKKUS_m_raw["Time"].dt.to_period("M").astype(str)

# Conversion to USXS spot exchange rate 
EXUSSKK_m_raw = EXSKKUS_m_raw.copy()
EXUSSKK_m_raw["USD-SKK Spot Exchange Rate"] = 1 / EXUSSKK_m_raw["SKK-USD Spot Exchange Rate"]
EXUSSKK_m_raw = EXUSSKK_m_raw.drop(columns=["SKK-USD Spot Exchange Rate"])

EXUSSKK_m_raw["Country"] = "SE"
EXUSSKK_m_raw = EXUSSKK_m_raw[["Country", "Time", "USD-SKK Spot Exchange Rate"]]

EXUSSKK_m_raw.tail()

,Country,Time,USD-SKK Spot Exchange Rate
653,SE,2025-06,0.104732
654,SE,2025-07,0.104194
655,SE,2025-08,0.104412
656,SE,2025-09,0.106693
657,SE,2025-10,0.106036


In [34]:
# US-DKK exchange rate, U.S. Dollar to One Danish Krone (initially DN-US converted to US-DN, monthly non-seasonally adjusted, 1971-01, 2025-09)
# https://fred.stlouisfed.org/series/EXDNUS
EXDKKUS_m_raw = fetch_FRED("EXDNUS") 
EXDKKUS_m_raw = EXDKKUS_m_raw.rename(columns= 
        {"date": "Time", 
         "EXDNUS": "DKK-USD Spot Exchange Rate"
})

EXDKKUS_m_raw["Time"] = EXDKKUS_m_raw["Time"].dt.to_period("M").astype(str)

# Conversion to USDN spot exchange rate 
EXUSDKK_m_raw = EXDKKUS_m_raw.copy()
EXUSDKK_m_raw["USD-DKK Spot Exchange Rate"] = 1 / EXUSDKK_m_raw["DKK-USD Spot Exchange Rate"]
EXUSDKK_m_raw = EXUSDKK_m_raw.drop(columns=["DKK-USD Spot Exchange Rate"])

EXUSDKK_m_raw["Country"] = "DK"
EXUSDKK_m_raw = EXUSDKK_m_raw[["Country", "Time", "USD-DKK Spot Exchange Rate"]]

EXUSDKK_m_raw.tail()

,Country,Time,USD-DKK Spot Exchange Rate
652,DK,2025-05,0.151110
653,DK,2025-06,0.154607
654,DK,2025-07,0.156387
655,DK,2025-08,0.156057
656,DK,2025-09,0.157257


In [33]:
# US-CZK exchange rate, U.S. Dollar to One Czech koruna (monthly non-seasonally adjusted, 1991-01, 2025-09)
# https://fred.stlouisfed.org/series/CCUSMA02CZM618N 
EXUSCZK_m_raw = fetch_FRED("CCUSMA02CZM618N") 
EXUSCZK_m_raw = EXUSCZK_m_raw.rename(columns= 
        {"date": "Time", 
         "CCUSMA02CZM618N": "USD-CZK Spot Exchange Rate"
})

EXUSCZK_m_raw["Time"] = EXUSCZK_m_raw["Time"].dt.to_period("M").astype(str)
EXUSCZK_m_raw["Country"] = "CZ"
EXUSCZK_m_raw = EXUSCZK_m_raw[["Country", "Time", "USD-CZK Spot Exchange Rate"]]

EXUSCZK_m_raw.tail()

,Country,Time,USD-CZK Spot Exchange Rate
412,CZ,2025-05,22.083318
413,CZ,2025-06,21.539476
414,CZ,2025-07,21.087739
415,CZ,2025-08,21.079476
416,CZ,2025-09,20.752818


In [32]:
# US-HUF exchange rate, U.S. Dollar to One Hungarian Forint (monthly non-seasonally adjusted, 1968-01, 2025-09)
# https://fred.stlouisfed.org/series/CCUSMA02HUM618N
EXUSHUF_m_raw = fetch_FRED("CCUSMA02HUM618N") 
EXUSHUF_m_raw = EXUSHUF_m_raw.rename(columns= 
        {"date": "Time", 
         "CCUSMA02HUM618N": "USD-HUF Spot Exchange Rate"
})

EXUSHUF_m_raw["Time"] = EXUSHUF_m_raw["Time"].dt.to_period("M").astype(str)
EXUSHUF_m_raw["Country"] = "HU"
EXUSHUF_m_raw = EXUSHUF_m_raw[["Country", "Time", "USD-HUF Spot Exchange Rate"]]

EXUSHUF_m_raw.tail()

,Country,Time,USD-HUF Spot Exchange Rate
688,HU,2025-05,357.905909
689,HU,2025-06,349.176667
690,HU,2025-07,341.577391
691,HU,2025-08,340.911905
692,HU,2025-09,333.840909


In [29]:
# US-PLN exchange rate, U.S. Dollar to One Polish Zloty (monthly non-seasonally adjusted, 1957-01, 2025-09)
# https://fred.stlouisfed.org/series/CCUSMA02PLM618N
EXUSPLN_m_raw = fetch_FRED("CCUSMA02PLM618N") 
EXUSPLN_m_raw = EXUSPLN_m_raw.rename(columns= 
        {"date": "Time", 
         "CCUSMA02PLM618N": "USD-PLN Spot Exchange Rate"
})

EXUSPLN_m_raw["Time"] = EXUSPLN_m_raw["Time"].dt.to_period("M").astype(str)
EXUSPLN_m_raw["Country"] = "PL"
EXUSPLN_m_raw = EXUSPLN_m_raw[["Country", "Time", "USD-PLN Spot Exchange Rate"]]

EXUSPLN_m_raw.tail()

,Country,Time,USD-PLN Spot Exchange Rate
820,PL,2025-05,3.769886
821,PL,2025-06,3.703057
822,PL,2025-07,3.640478
823,PL,2025-08,3.666586
824,PL,2025-09,3.629195


In [31]:
# US-RON exchange rate, U.S. Dollar to One Romanian Leu (monthly non-seasonally adjusted, 1960-01, 2023-11)
# https://fred.stlouisfed.org/series/CCUSMA02PLM618N
EXUSRON_m_raw = fetch_FRED("ROUCCUSMA02STM") 
EXUSRON_m_raw = EXUSRON_m_raw.rename(columns= 
        {"date": "Time", 
         "ROUCCUSMA02STM": "USD-RON Spot Exchange Rate"
})

EXUSRON_m_raw["Time"] = EXUSRON_m_raw["Time"].dt.to_period("M").astype(str)
EXUSRON_m_raw["Country"] = "RO"
EXUSRON_m_raw = EXUSRON_m_raw[["Country", "Time", "USD-RON Spot Exchange Rate"]]

EXUSRON_m_raw.tail()

,Country,Time,USD-RON Spot Exchange Rate
798,RO,2023-07,4.4689
799,RO,2023-08,4.5294
800,RO,2023-09,4.6470
801,RO,2023-10,4.7027
802,RO,2023-11,4.6035


In [38]:
# Crude Oil Prices: Brent - Europe (monthly, average price, Dollars per barrel, not seasonally adjusted, 1987-01, 2025-09)
# https://fred.stlouisfed.org/series/MCOILBRENTEU

oilprice_m_raw = fetch_FRED("MCOILBRENTEU") 
oilprice_m_raw = oilprice_m_raw.rename(columns= 
        {"date": "Time", 
         "MCOILBRENTEU": "Crude Oil Price (Brent, Europe)"
})

oilprice_m_raw["Time"] = oilprice_m_raw["Time"].dt.to_period("M").astype(str)

oilprice_m_raw.tail()

,Time,"Crude Oil Price (Brent, Europe)"
456,2025-05,64.45
457,2025-06,71.44
458,2025-07,71.04
459,2025-08,67.87
460,2025-09,67.99


In [39]:
# Nominal Broad U.S. Dollar Index (monthly, index Jan 2006=100, not seasonally adjusted, 2006-01, 2025-09)
# https://fred.stlouisfed.org/series/TWEXBGSMTH

usdi_m_raw = fetch_FRED("TWEXBGSMTH") 
usdi_m_raw = usdi_m_raw.rename(columns= 
        {"date": "Time", 
         "TWEXBGSMTH": "Nominal Broad USD Index"
})

usdi_m_raw["Time"] = usdi_m_raw["Time"].dt.to_period("M").astype(str)

usdi_m_raw.tail()

,Time,Nominal Broad USD Index
626,2025-05,122.6869
627,2025-06,120.9747
628,2025-07,120.5266
629,2025-08,120.9844
630,2025-09,120.4534


In [42]:
# Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity, Quoted on an Investment Basis  (from daily to monthly, percent, not seasonally adjusted, 1962-01, 2025-10)
# https://fred.stlouisfed.org/series/DGS10

ustyield_d_raw = fetch_FRED("DGS10") 
ustyield_d_raw = ustyield_d_raw.rename(columns= 
        {"date": "Time", 
         "DGS10": "Market Yield on 10-Year US Trasury Securities"
})

# Aggregation (dimension from daily to monthly)
ustyield_d_raw = ustyield_d_raw.set_index("Time")
ustyield_m_raw = ustyield_d_raw.resample("M").mean().reset_index()
ustyield_m_raw["Time"] = ustyield_m_raw["Time"].dt.to_period("M").astype(str)

ustyield_m_raw.tail()

,Time,Market Yield on 10-Year US Trasury Securities
761,2025-06,4.383500
762,2025-07,4.391818
763,2025-08,4.264762
764,2025-09,4.120476
765,2025-10,4.062353


#### **3.5) OTHERS-extracted indicators**

In [13]:
# Tariffs ratio

### **4) DATA ASSEMBLING**

#### **4.3) GLOBAL CONTROLS DF**

In [53]:
# Aggregate Global Control df
# Merging Index: Time
# Time frame: 2006/01 - 2025/09
# (!!!) To run the model we need no NaN in the df, so we dop them, causing a shrinking of the df time range
global_control_df = (oilprice_m_raw.merge(usdi_m_raw, on="Time", how="outer").merge(ustyield_m_raw, on="Time", how="outer").sort_values("Time").reset_index(drop=True))
global_control_df = global_control_df.dropna()

# Uploading in aggregate_df
global_control_df.to_csv("aggregate_df/global_control_df.csv", index=False)

global_control_df.tail()

,Time,"Crude Oil Price (Brent, Europe)",Nominal Broad USD Index,Market Yield on 10-Year US Trasury Securities
760,2025-05,64.45,122.6869,4.423810
761,2025-06,71.44,120.9747,4.383500
762,2025-07,71.04,120.5266,4.391818
763,2025-08,67.87,120.9844,4.264762
764,2025-09,67.99,120.4534,4.120476


#### **4.4) COUNTRY-SPECIFIC CONTROLS DF**

In [52]:
# Aggregate Country-Specific Control df
# Merging Index: Time, Country
# Time frame: 
# (!!!) To run the model we need no NaN in the df, so we dop them, causing a shrinking of the df time range
country_control_df = (oilprice_m_raw.merge(usdi_m_raw, on="Time", how="outer").merge(ustyield_m_raw, on="Time", how="outer").sort_values("Time").reset_index(drop=True))
country_control_df = country_control_df.dropna()

# Uploading in aggregate_df
country_control_df.to_csv("aggregate_df/country_control_df.csv", index=False)

country_control_df.tail()

,Time,"Crude Oil Price (Brent, Europe)",Nominal Broad USD Index,Market Yield on 10-Year US Trasury Securities
760,2025-05,64.45,122.6869,4.423810
761,2025-06,71.44,120.9747,4.383500
762,2025-07,71.04,120.5266,4.391818
763,2025-08,67.87,120.9844,4.264762
764,2025-09,67.99,120.4534,4.120476
